# This notebook allow users to pull NOAA Coastal Ocean ReAnalysis (CORA) data that is stored on NOAA's Open Data Dissemination (NODD) service and create a 2D plot of water levels.

In [1]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import dask
import intake
import xarray as xr
import holoviews as hv
import geoviews as gv
import hvplot.xarray
import holoviews.operation.datashader as dshade
import cmocean

hv.extension('bokeh')

C:\Users\John.Ratcliff\AppData\Local\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


**Access the data on the NODD and initialize the available CORA datasets.** 

*This accesses a .yml file located on the NODD that shows which CORA output files are available to import.*

In [2]:

catalog = intake.open_catalog("s3://noaa-nos-cora-pds/CORA_intake.yml",storage_options={'anon':True})
list(catalog)


['CORA-V1-fort.63',
 'CORA-V1-maxele.63',
 'CORA-V1-fort.64',
 'CORA-V1-500m-grid-1979-2022']

**The CORA-V1-fort.63 file contains hourly water levels at the model mesh nodes for 1979-2022. The CORA-V1-maxele.63 file contains the maximum water level for the entire 1979-2022 period modeled at each of the model nodes. The CORA-V1-fort.64 file contains the hourly current velocities (u and v) at each of the model nodes for 1979-2022. The CORA-V1-500m-grid-1979-2022 file contains hourly water levels that have been interpolated from the model mesh nodes to uniformly space 500-meter grid nodes.**

## Using to_dask() with one of the catalog files will create an xarray dataset that contains the model grid information along with the time and water level ('zeta') variables.

**This is the dataset for the hourly water levels at all model nodes.**

In [3]:
ds_hourly = catalog["CORA-V1-fort.63"].to_dask()
ds_hourly

<xarray.Dataset>
Dimensions:      (mesh: 1, node: 1813443, nele: 3564104, nvertex: 3, nbou: 186,
                  nope: 1, neta_dim: 186, nvel_dim: 62972, time: 385704)
Coordinates:
  * time         (time) datetime64[ns] 1979-01-01 ... 2022-12-31T23:00:00
    x            (node) float64 dask.array<chunksize=(160000,), meta=np.ndarray>
    y            (node) float64 dask.array<chunksize=(160000,), meta=np.ndarray>
Dimensions without coordinates: mesh, node, nele, nvertex, nbou, nope,
                                neta_dim, nvel_dim
Data variables: (12/14)
    adcirc_mesh  (mesh) int32 dask.array<chunksize=(1,), meta=np.ndarray>
    depth        (node) float64 dask.array<chunksize=(160000,), meta=np.ndarray>
    element      (nele, nvertex) int32 dask.array<chunksize=(3564104, 3), meta=np.ndarray>
    ibtype       (nbou) int32 dask.array<chunksize=(186,), meta=np.ndarray>
    ibtypee      (nope) int32 dask.array<chunksize=(1,), meta=np.ndarray>
    max_nvdll    float64 ...
    ...           ...
    nbvv         (nvel_dim) int32 dask.array<chunksize=(62972,), meta=np.ndarray>
    neta         float64 ...
    nvdll        (nope) int32 dask.array<chunksize=(1,), meta=np.ndarray>
    nvel         float64 ...
    nvell        (nbou) int32 dask.array<chunksize=(186,), meta=np.ndarray>
    zeta         (time, node) float64 dask.array<chunksize=(240, 160000), meta=np.ndarray>
Attributes: (12/49)
    Conventions:        UGRID-0.9.0
    NCO:                netCDF Operators version 4.7.5 (Homepage = http://nco...
    a00:                0.0
    agrid:              NOMAD mesh v1e MSL - fixed bahamas bathy
    b00:                1.0
    c00:                0.0
    ...                 ...
    slam0:              265.5
    source:             RENCI_HPC
    statim:             0.0
    tau0:               -3.0
    title:              NOAA-Reanalysis-Post
    version:            noaa.stofs.2d.glo.v1.1.0r2-21-g7052115

**This is the dataset for the maximum water levels at all model nodes for each of the simulated years.**

In [4]:

ds_max = catalog["CORA-V1-maxele.63"].to_dask()
ds_max


C:\Users\John.Ratcliff\AppData\Local\anaconda3\Lib\site-packages\xarray\core\dataset.py:265: UserWarning: The specified chunks separate the stored chunks along dimension "node" starting at index 160000. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(


<xarray.Dataset>
Dimensions:           (mesh: 1, node: 1813443, nele: 3564104, nvertex: 3,
                       nbou: 186, nope: 1, neta_dim: 186, nvel_dim: 62972,
                       time: 44)
Coordinates:
  * time              (time) datetime64[ns] 1980-01-05 1981-01-04 ... 2023-01-05
    x                 (node) float64 dask.array<chunksize=(160000,), meta=np.ndarray>
    y                 (node) float64 dask.array<chunksize=(160000,), meta=np.ndarray>
Dimensions without coordinates: mesh, node, nele, nvertex, nbou, nope,
                                neta_dim, nvel_dim
Data variables: (12/15)
    adcirc_mesh       (mesh) int32 dask.array<chunksize=(1,), meta=np.ndarray>
    depth             (node) float64 dask.array<chunksize=(160000,), meta=np.ndarray>
    element           (nele, nvertex) int32 dask.array<chunksize=(3564104, 3), meta=np.ndarray>
    ibtype            (nbou) int32 dask.array<chunksize=(186,), meta=np.ndarray>
    ibtypee           (nope) int32 dask.array<chunksize=(1,), meta=np.ndarray>
    max_nvdll         float64 ...
    ...                ...
    neta              float64 ...
    nvdll             (nope) int32 dask.array<chunksize=(1,), meta=np.ndarray>
    nvel              float64 ...
    nvell             (nbou) int32 dask.array<chunksize=(186,), meta=np.ndarray>
    time_of_zeta_max  (time, node) float64 dask.array<chunksize=(44, 160000), meta=np.ndarray>
    zeta_max          (time, node) float64 dask.array<chunksize=(44, 160000), meta=np.ndarray>
Attributes: (12/49)
    Conventions:        UGRID-0.9.0
    NCO:                netCDF Operators version 4.7.5 (Homepage = http://nco...
    a00:                0.0
    agrid:              NOMAD mesh v1e MSL - fixed bahamas bathy
    b00:                1.0
    c00:                0.0
    ...                 ...
    slam0:              265.5
    source:             RENCI_HPC
    statim:             0.0
    tau0:               -3.0
    title:              NOAA-Reanalysis-Post
    version:            noaa.stofs.2d.glo.v1.1.0r2-21-g7052115

## A function is created that will create a 2D plot of the zeta_max variable from the dataset.

In [5]:

def plot_gridded_output(title, grid, var, color_map, width, height):
    dshade.datashade.precompute = True
    v = np.vstack((grid.x,grid.y, var)).T
    verts = pd.DataFrame(v, columns=['x','y','var'])
    points = gv.operation.project_points(gv.Points(verts, vdims=['var']))
    tris = pd.DataFrame(grid['element'].values.astype('int')-1, columns=['v0','v1','v2'])
    tiles = gv.tile_sources.OSM
    trimesh = gv.TriMesh((tris, points), label=title)
    mesh = dshade.rasterize(trimesh).opts(cmap=color_map, colorbar=True, width=width, height=height)
    return tiles * mesh
    

## Create a variable for an annual maximum water elevation and plot using the function.

In [6]:

max_annual_wl = ds_max["zeta_max"].isel(time=43)


In [7]:

plot_gridded_output('Max Water Elevation (ref. to msl) for 2022', ds_max, max_annual_wl, cmocean.cm.balance, 1000, 700)


:DynamicMap   []
   :Overlay
      .WMTS.I                                                                                     :WMTS   [Longitude,Latitude]
      .Image.Max_Water_Elevation_left_parenthesis_ref_full_stop_to_msl_right_parenthesis_for_2022 :Image   [x,y]   (x_y var)

## Create a variable for the maximum water elevation over all available years and plot.

In [8]:

max_total_wl = ds_max['zeta_max'].max(dim='time').compute()


In [9]:

plot_gridded_output('Max Water Elevation (ref. to msl) for 1979-2022', ds_hourly, max_total_wl, cmocean.cm.balance, 1000, 700)


:DynamicMap   []
   :Overlay
      .WMTS.I                                                                                                       :WMTS   [Longitude,Latitude]
      .Image.Max_Water_Elevation_left_parenthesis_ref_full_stop_to_msl_right_parenthesis_for_1979_hyphen_minus_2022 :Image   [x,y]   (x_y var)